In [54]:

import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset


import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

import easydict

In [55]:
#데이터 불러오기
train = pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\JobCare_data\\train.csv')
test = pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\JobCare_data\\test.csv')
d_code=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\Jobcare_data\\속성_D_코드.csv',index_col=0).T.to_dict()
h_code=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\Jobcare_data\\속성_H_코드.csv',index_col=0).T.to_dict()
l_code=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\Jobcare_data\\속성_L_코드.csv',index_col=0).T.to_dict()

In [56]:
#데이터 재가공
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    
    return df


train = add_code(train, d_code, h_code, l_code)
test = add_code(test, d_code, h_code, l_code)

In [61]:
train = train.drop(['id', 'contents_open_dt','person_rn', 'contents_rn'], axis=1) 
test = test.drop(['id', 'contents_open_dt','person_rn', 'contents_rn'], axis=1)

# print(train.head())
train['d_l_match_yn']=train['d_l_match_yn'].replace([True,False],[1,0])
train['d_m_match_yn']=train['d_m_match_yn'].replace([True,False],[1,0])
train['d_s_match_yn']=train['d_s_match_yn'].replace([True,False],[1,0])
train['h_l_match_yn']=train['h_l_match_yn'].replace([True,False],[1,0])
train['h_m_match_yn']=train['h_m_match_yn'].replace([True,False],[1,0])
train['h_s_match_yn']=train['h_s_match_yn'].replace([True,False],[1,0])

test['d_l_match_yn']=test['d_l_match_yn'].replace([True,False],[1,0])
test['d_m_match_yn']=test['d_m_match_yn'].replace([True,False],[1,0])
test['d_s_match_yn']=test['d_s_match_yn'].replace([True,False],[1,0])
test['h_l_match_yn']=test['h_l_match_yn'].replace([True,False],[1,0])
test['h_m_match_yn']=test['h_m_match_yn'].replace([True,False],[1,0])
test['h_s_match_yn']=test['h_s_match_yn'].replace([True,False],[1,0])

In [62]:
#컬럼 순서 변경
train.columns
train=train[['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_f', 'person_prefer_g',
       'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_prefer_d_1_n',
       'person_prefer_d_1_s', 'person_prefer_d_1_m', 'person_prefer_d_1_l',
       'person_prefer_d_2_n', 'person_prefer_d_2_s', 'person_prefer_d_2_m',
       'person_prefer_d_2_l', 'person_prefer_d_3_n', 'person_prefer_d_3_s',
       'person_prefer_d_3_m', 'person_prefer_d_3_l', 'contents_attribute_d_n',
       'contents_attribute_d_s', 'contents_attribute_d_m',
       'contents_attribute_d_l', 'person_prefer_h_1_m', 'person_prefer_h_2_m',
       'person_prefer_h_3_m', 'contents_attribute_h_m', 'person_prefer_h_1_l',
       'person_prefer_h_2_l', 'person_prefer_h_3_l', 'contents_attribute_h_l',
       'contents_attribute_l_n', 'contents_attribute_l_s',
       'contents_attribute_l_m', 'contents_attribute_l_l','target']]

test=test[['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_f', 'person_prefer_g',
       'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_prefer_d_1_n',
       'person_prefer_d_1_s', 'person_prefer_d_1_m', 'person_prefer_d_1_l',
       'person_prefer_d_2_n', 'person_prefer_d_2_s', 'person_prefer_d_2_m',
       'person_prefer_d_2_l', 'person_prefer_d_3_n', 'person_prefer_d_3_s',
       'person_prefer_d_3_m', 'person_prefer_d_3_l', 'contents_attribute_d_n',
       'contents_attribute_d_s', 'contents_attribute_d_m',
       'contents_attribute_d_l', 'person_prefer_h_1_m', 'person_prefer_h_2_m',
       'person_prefer_h_3_m', 'contents_attribute_h_m', 'person_prefer_h_1_l',
       'person_prefer_h_2_l', 'person_prefer_h_3_l', 'contents_attribute_h_l',
       'contents_attribute_l_n', 'contents_attribute_l_s',
       'contents_attribute_l_m', 'contents_attribute_l_l']]


In [63]:
torch.manual_seed(42)


train.shape
test.shape

(46404, 58)

In [115]:

x = train.iloc[0:100000, :-1]
y = train.iloc[0:100000, -1]

# print(x)
# print(y)


x= x.to_numpy()
print(x[0])
print(y[0])
x=np.where(x > 1000, x/1000, x)
x=np.where(x > 100, x/100, x)
x=np.where(x > 10, x/10, x)
# x=x/100
x=torch.Tensor(x)
print(x.size())
print(x[0][1])
y=torch.Tensor(y)
# print(y)


dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)


x_test=test.iloc[0:1000]
x_test=x_test.to_numpy()
x_test=np.where(x_test > 1000, x_test/1000, x_test)
x_test=np.where(x_test > 100, x_test/100, x_test)
x_test=np.where(x_test > 10, x_test/10, x_test)
x_test=x_test/100

print(x_test.shape)
x_test=torch.FloatTensor(x_test)

[   1    1    1    0    0    0    1    4    3    5  275  370  369    8
    1    1    4   95   59    3    3   10    2    1    2 1608  275    1
    4  139  275  274  274  216  369  368  297  216  369  368  297  216
  275  274  274  216  316  398  368  422    3   94   58   94 1607 1606
 1605 2016]
1
torch.Size([100000, 58])
tensor(1.)
(1000, 58)


In [109]:
class LogisticRegression(nn.Module):
    
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(58,16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )        

    def forward(self, x):
        return self.linear(x)

model1=LogisticRegression()

In [99]:

class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(58, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

model2 = BinaryClassifier()

In [110]:
optimizer1 = optim.SGD(model1.parameters(), lr=0.001)
print(model1.parameters)

optimizer2 = optim.SGD(model2.parameters(),lr=0.001)
print(model2.parameters)

<bound method Module.parameters of LogisticRegression(
  (linear): Sequential(
    (0): Linear(in_features=58, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=1, bias=True)
    (4): Sigmoid()
  )
)>
<bound method Module.parameters of BinaryClassifier(
  (linear): Linear(in_features=58, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)>


In [116]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):

        x_train, y_train = samples
        # H(x) 계산
        hypothesis = model1(x_train)
        
        
    
        # cost 계산
        # print(y_train.size())
        hypothesis=hypothesis.squeeze()
        # print(hypothesis.size())
        # print(hypothesis)
        # print(y_train)
        
        cost = F.binary_cross_entropy(hypothesis, y_train)

        # cost로 H(x) 개선
        optimizer1.zero_grad()
        cost.backward()
        optimizer1.step()

        # 20번마다 로그 출력
        if epoch % 10 == 0:
            prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
            correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
            accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
            print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
                epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item(), accuracy * 100,
            ))



Epoch    0/1000 Batch 1/391 Cost: 0.650749 Accuracy 62.11%
Epoch    0/1000 Batch 2/391 Cost: 0.645093 Accuracy 64.06%
Epoch    0/1000 Batch 3/391 Cost: 0.644172 Accuracy 65.62%
Epoch    0/1000 Batch 4/391 Cost: 0.661180 Accuracy 61.72%
Epoch    0/1000 Batch 5/391 Cost: 0.656204 Accuracy 58.59%
Epoch    0/1000 Batch 6/391 Cost: 0.685742 Accuracy 59.38%
Epoch    0/1000 Batch 7/391 Cost: 0.629340 Accuracy 64.06%
Epoch    0/1000 Batch 8/391 Cost: 0.676561 Accuracy 61.33%
Epoch    0/1000 Batch 9/391 Cost: 0.671597 Accuracy 59.38%
Epoch    0/1000 Batch 10/391 Cost: 0.642452 Accuracy 61.72%
Epoch    0/1000 Batch 11/391 Cost: 0.643757 Accuracy 61.33%
Epoch    0/1000 Batch 12/391 Cost: 0.641670 Accuracy 61.72%
Epoch    0/1000 Batch 13/391 Cost: 0.664180 Accuracy 59.38%
Epoch    0/1000 Batch 14/391 Cost: 0.649645 Accuracy 61.72%
Epoch    0/1000 Batch 15/391 Cost: 0.673566 Accuracy 59.77%
Epoch    0/1000 Batch 16/391 Cost: 0.680087 Accuracy 61.72%
Epoch    0/1000 Batch 17/391 Cost: 0.631952 Accur

KeyboardInterrupt: 

In [53]:
# 테스트 데이터를 사용하여 모델을 테스트한다.

# with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
#     X_test = x_test.to(device)
#     Y_test = y_test.to(device)

#     prediction = model1(X_test)
#     correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
#     accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산

#     print('Accuracy:', accuracy.item())


torch.Size([1000, 58])
<class 'torch.Tensor'>


0      1
1      1
2      0
3      0
4      0
      ..
227    0
228    0
229    0
230    0
231    0
Length: 232, dtype: int64

In [ ]:
result=pd.read_csv('/content/sample_submission.csv')

test_data=test.iloc[:]
hypothesis=model1(test_data)

prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
prediction=pd.Series(prediction)
prediction=prediction.replace([True,False],[1,0])

result['target']=prediction
result.to_csv('result.csv',index=False)

